#### Change Working Directory

In [420]:
path = 'C:\\zzzzdownload\\'
os.chdir(path)
#os.getcwd()

#### Load File 
The Total of Order line items (Transactions) is 6727

In [153]:
import pandas as pd
df = pd.read_csv('interview_sample_data.csv')
total_items = len(df)
#df

#### Add column for original_order_id
Original_order_id will be filled with the order_id if type is sale. For type return it will get the parent_order_id if the return is valid. 

The invalid returns are orphans whose parent_item is not in the dataframe and therefore it's impossible to get their parent's order_id. Orphans will get a -1 in the original_order_id column. 

In [194]:
new_empty_column = [0] * total_items
original_order_id = new_empty_column

In [201]:
for index, row in df.iterrows():
    
    # if item is type sale use order_id
    if row['quantity'] == 1:
        value = row['order_id']
    
    # if item is type return
    else:
        
        # look for the parent line
        parent_line = df.loc[df['line_id'] == row['parent_line_id'] ]
        
        # parent line is not available in the dataframe -> set 0
        if(len(parent_line)) == 0:
            value = -1
        
        # use line if available
        else:
            value = parent_line['order_id'].iloc[0]
    
    original_order_id[index] = value

df['original_order_id'] = original_order_id
# original_order_id
#df


#### Remove Orphans
The 35 orphans will be placed in a separate dataframe and the valid rows will be also stored in a valid dataframe for continuing the analysis.

In [204]:
df_orphans = df.loc[df['original_order_id'] == -1 ]
df_valid = df.loc[df['original_order_id'] != -1 ]
#len(df_orphans)
#len(df_valid)

#### Total Orders
There are 5470 total orders, every order has between 8 and 1 line item.
After reving the orphan records, there are 5447 remaining orders.

In [217]:
all_orders = df.order_id
valid_orders = df_valid.order_id
#len(all_orders.value_counts())
#len(valid_orders.value_counts())


#### Find Order Type

In [244]:
order_ids = valid_orders.tolist()
order_types = {}
order_types_by_original = {}

for order_id in order_ids:
    
    # get related items
    items = df_valid.loc[df_valid['order_id'] == order_id ]
    items_by_original = df_valid.loc[df_valid['original_order_id'] == order_id ]
    
    # ----- FOR ORDERS
    # count types of items
    total_sale = 0
    total_return = 0
    for index, row in items.iterrows():
        if row.quantity == 1:
            total_sale += 1
        else:
            total_return +=1
    
    # decide the order's type base of the item's type
    order_type = 1  # most are sales
    if total_sale > 0 and total_return > 0:
        order_type = 3  # exchange
    elif total_sale == 0 and total_return > 1:
        order_type = 2 # returns
        
    # save type to dict
    order_types[row['order_id']] = order_type
    
    # ----- FOR ORIGINAL ORDERS
    # count types of items
    total_sale = 0
    total_return = 0
    for index, row in items_by_original.iterrows():
        if row.quantity == 1:
            total_sale += 1
        else:
            total_return +=1
    
    # decide the order's type base of the item's type
    order_type = 1  # most are sales
    if total_sale > 0 and total_return > 0:
        order_type = 3  # exchange
    elif total_sale == 0 and total_return > 1:
        order_type = 2 # returns
        
    # save type to dict
    order_types_by_original[row['order_id']] = order_type

        
    

In [250]:
# move result from dict to list
order_types_list = []
order_types_original_list = []

for index, row in df_valid.iterrows():
    
    # get type for current item
    order_type = order_types.get(row['order_id'])
    order_type_original = order_types_by_original.get(row['original_order_id'])
    
    # save to list
    order_types_list.append(order_type)
    order_types_original_list.append(order_type_original)


In [280]:
# save to dataframe
df_valid_copy = df_valid.copy()
df_valid_copy['order_type'] = order_types_list
df_valid_copy['order_type_original'] = order_types_original_list


len(df_valid_copy.loc[df_valid_copy['order_type_original'] == 1])

6385

# Final Results

#### Export Results

In [304]:
df_valid_copy.to_csv('results.csv')
# df_valid_copy

#### Total Count for order types
According to the order of each item, there are a total of 6109 orders for sale, 43 orders for returns and 540 orders for exchange. The total of orders is 6692.

In [305]:
sales = df_valid_copy.loc[df_valid_copy['order_type'] == 1]
returns = df_valid_copy.loc[df_valid_copy['order_type'] == 2]
exchange = df_valid_copy.loc[df_valid_copy['order_type'] == 3]
# len(sales)
# len(returns)
# len(exchange)

#### Total Count for order types by original
According to the original order, there are a total of 6385 orders for sale and 307 orders for exchange. There are 0 orders for returns because returns with no matching order where previously extracted as invalid orphan data (35 records). The total of orders is 6692.

In [306]:
sales_o = df_valid_copy.loc[df_valid_copy['order_type_original'] == 1]
returns_o = df_valid_copy.loc[df_valid_copy['order_type_original'] == 2]
exchange_o = df_valid_copy.loc[df_valid_copy['order_type_original'] == 3]
# len(sales_o)
# len(returns_o)
# len(exchange_o)

#### Total Income
The total income is 580,899.98

In [336]:
df_valid_copy['line_total'].agg(['sum'])

sum    580899.98
Name: line_total, dtype: float64

#### Total Sales and returns
Total income from sales is 642,367.80 and total loss from returns is -61,467.82. The 9.56% of money the clients spend is lost due to returns.

The total number of sales is 6,129 and the total number of returns is 563. The 9.185% of items are returned after purchase.

In [338]:
df_valid_copy.groupby('quantity').agg(['sum', 'count'])['line_total']

,sum,count
quantity,,
-1,-61467.82,563
1,642367.80,6129


#### Sales by stores
Total number and total amount in sales by store.

In [404]:
sales_items = df_valid_copy.loc[df_valid_copy['quantity'] == 1]
sales_by_store = sales_items.groupby('store_id').agg(['sum', 'count'])['line_total']
sales_by_store = sales_by_store.rename({'sum': 'sum_sales', 'count': 'count_sales'}, axis='columns')
sales_by_store

,sum_sales,count_sales
store_id,,
5,162412.00,1502
11,64014.53,627
25,80093.23,706
28,122991.82,1224
33,111246.31,1125
34,101609.91,945


#### Returns by stores
In the case of returns the store_id has no importance because the return should be accounted to the store where the item was initially sold. So, a new column is to be created with the parent_store.

In [395]:
# select only returns
return_items = df_valid_copy.loc[df_valid_copy['quantity'] == -1]

# find store_id for each parent_line_id
parent_store_ids = []
for index, row in return_items.iterrows():
    parent_store_id = df_valid_copy.loc[df_valid_copy['line_id'] == row['parent_line_id']]
    parent_store_ids.append(int(parent_store_id.store_id))

# save results in a new dataframe
return_items_copy = return_items.copy()
return_items_copy['parent_store_id'] = parent_store_ids
# return_items_copy


In [403]:
returns_by_store = return_items_copy.groupby('parent_store_id').agg(['sum', 'count'])['line_total']
returns_by_store = returns_by_store.rename({'sum': 'sum_returns', 'count': 'count_returns'}, axis='columns')
returns_by_store

,sum_returns,count_returns
parent_store_id,,
5,-15762.59,146
11,-4835.82,46
25,-8596.21,71
28,-10879.98,102
33,-8840.11,96
34,-12553.11,102


In [414]:
concat = pd.concat([sales_by_store, returns_by_store], axis = 1)
concat = concat.rename({'sum': 'X', 'count': 'Y'}, axis='columns')
concat.index.names = ['store_id']
concat

,sum_sales,count_sales,sum_returns,count_returns
store_id,,,,
5,162412.00,1502,-15762.59,146
11,64014.53,627,-4835.82,46
25,80093.23,706,-8596.21,71
28,122991.82,1224,-10879.98,102
33,111246.31,1125,-8840.11,96
34,101609.91,945,-12553.11,102


In [421]:
concat["return_rate"] = concat.count_returns / concat.count_sales * 100
concat

,sum_sales,count_sales,sum_returns,count_returns,return_rate
store_id,,,,,
5,162412.00,1502,-15762.59,146,9.720373
11,64014.53,627,-4835.82,46,7.336523
25,80093.23,706,-8596.21,71,10.056657
28,122991.82,1224,-10879.98,102,8.333333
33,111246.31,1125,-8840.11,96,8.533333
34,101609.91,945,-12553.11,102,10.793651
